In [2]:
import random as rng
from itertools import product
from collections import Counter

<h3> DEFINICJA OBIEKTU KARTA </h3>
<p> karta może być figurą od 2 do A, i mieć kolor:<br> [H]eart [D]iamonds [C]lubs [S]pades<br> KIER___KARO_____TREFL__PIK</p>

<p>kazda figura ma przypisaną rangę (self.rank) od 0 do 12. <br>Rangi pozwalają na sortowanie kart</p>

In [3]:
class Card:
    def __init__(self,fig,col):
        self.slownik =[
            {
            '2' : '2',
            '3' : '3',
            '4' : '4',
            '5' : '5',
            '6' : '6',
            '7' : '7',
            '8' : '8',
            '9' : '9',
            '10' : '10',
            'J' : 'Jack',
            'Q' : 'Queen',
            'K' : 'King',
            'A' : 'Ace'},
            {
            'H': 'Hearts',
            'D': 'Diamonds',
            'C': 'Clubs',
            'S': 'Spades'}
            ]
        
        
        while fig.upper() not in self.slownik[0].keys():
            print('błędne oznaczenie figury karty')
            fig = input('Podaj poprawne oznaczenie figury karty: ')
        self.figure = fig.upper()
        
        while col.upper() not in self.slownik[1].keys():
            print('błędne oznaczenie koloru karty')
            col = input('Podaj poprawne oznaczenie koloru karty: ')
        self.colour = col.upper()
        

        rank_dict = {figure : rank for rank,figure in enumerate(self.slownik[0].keys())}
        self.rank_of_card = rank_dict[self.figure]

    def __enter__(self):
        return self

    def __str__(self):
        return f'{self.slownik[0][self.figure]} of {self.slownik[1][self.colour]}'
    
    def __repr__(self):
        return f'{self.figure}{self.colour}'

    def __lt__(self,other):
        return self.rank_of_card < other.rank_of_card
    def __le__(self,other):
        return self.rank_of_card <= other.rank_of_card
    def __gt__(self,other):
        return self.rank_of_card > other.rank_of_card
    def __ge__(self,other):
        return self.rank_of_card >= other.rank_of_card
    def __eq__(self,other):
        return self.rank_of_card == other.rank_of_card


<p>małe testy karty </p>

In [4]:
print(Card('A', 'D').__repr__())
print(Card('3', 'h') < Card('j','s'))

AD
True


<h3> DEFINICJA OBIEKTU TALIA </h3> 

In [5]:
class Deck:
    def __init__(self):
        figures = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
        colours = 'H D C S'.split()
        self.talia = []
        for i in product (figures, colours):
            self.talia.append(Card(i[0], i[1]))
    def __len__(self):
        return len(self.talia)
    
    def __enter__(self):
        return self

    def __str__(self):
        return f'karty w talii [{len(self)}]: \n{self.talia}\n * * * * *'
    def shuffle(self):
        temp_deck = self.talia[:]
        self.talia = []
        while temp_deck:
            self.talia.append(temp_deck.pop(rng.randint(0,len(temp_deck)-1)))
        #print(temp_deck)
    def draw(self, n = 1):
        card_list = []
        while n:
            card_list.append(self.talia.pop())
            n-=1
        return card_list
    def reset(self):
        self.__init__()
        self.shuffle()



małe testy talii

In [6]:
t = Deck()
print(t)
x = t.draw(10)
print(x)
print(len(x))
print(t)
print(len(t))

karty w talii [52]: 
[2H, 2D, 2C, 2S, 3H, 3D, 3C, 3S, 4H, 4D, 4C, 4S, 5H, 5D, 5C, 5S, 6H, 6D, 6C, 6S, 7H, 7D, 7C, 7S, 8H, 8D, 8C, 8S, 9H, 9D, 9C, 9S, 10H, 10D, 10C, 10S, JH, JD, JC, JS, QH, QD, QC, QS, KH, KD, KC, KS, AH, AD, AC, AS]
 * * * * *
[AS, AC, AD, AH, KS, KC, KD, KH, QS, QC]
10
karty w talii [42]: 
[2H, 2D, 2C, 2S, 3H, 3D, 3C, 3S, 4H, 4D, 4C, 4S, 5H, 5D, 5C, 5S, 6H, 6D, 6C, 6S, 7H, 7D, 7C, 7S, 8H, 8D, 8C, 8S, 9H, 9D, 9C, 9S, 10H, 10D, 10C, 10S, JH, JD, JC, JS, QH, QD]
 * * * * *
42


In [69]:
class Hand:
    def __init__(self):
        self.cards_in_hand = []
        self.high_card = None
        self.in_hand = {
            'pairs' : [],  # pairs
            'threes' : [], # threes
            'straights' : [], # straights
            'flush' : [], # flushes
            'full_house' : [], #full house
            'fours' : [], # fours of a kind
            'straight_flush' : [] # straight flushes
            }
    def get_card(self, talia: Deck, n :int):
        for card in talia.draw(n):
            self.cards_in_hand.append(card)
        self.find_high_card()
    
    def get_specific_cards(self, talia: Deck, list_of_cards: list):
        for card in list_of_cards:
            self.cards_in_hand.append(Card(card.upper()[0],card.upper()[1]))
            for id, card_in_deck in enumerate(talia.talia):
                if card_in_deck.figure == card.upper()[0] and card_in_deck.colour == card.upper()[1]:
                    del talia.talia[id]

    def find_high_card(self):
        self.high_card = (self.cards_in_hand[0])
        for card in self.cards_in_hand:
            if card.rank_of_card > self.high_card.rank_of_card:
                self.high_card = card

    def __str__(self):
        return f'cards in hand:\n{sorted(self.cards_in_hand)}\nHigh Card: {self.high_card}'

    def reset_hand(self):
        self.__init__()
    def check_hand(self):
        #print('**********')
        ranks = sorted([card.rank_of_card for card in self.cards_in_hand])
        if [ranks[i+1] - ranks[i] for i in range(0,4)] == [1,1,1,1]:
            self.in_hand['straights'] = ranks[-1]

        figures_in_hand = [card.rank_of_card for card in self.cards_in_hand]  
        #print(self.cards_in_hand)
        #print(f'{sorted(figures_in_hand)=}')
        ile_czego = list(Counter(sorted(figures_in_hand)).items())

        if len(set([card.colour for card in self.cards_in_hand])) == 1 :
            self.in_hand['flush'] = self.cards_in_hand[0].colour
            
        print(f'{ile_czego = }')
        while len(ile_czego):
            a = ile_czego.pop()
            if a[1] == 2:
                self.in_hand['pairs'].append(a[0])
            elif a[1] == 3:
                self.in_hand['threes'].append(a[0])
            elif a[1] == 4:
                self.in_hand['fours'].append(a[0])
        #print(self.in_hand)
        #print([q for q in self.in_hand.items() if q[1] != []])

In [72]:
ta = Deck()
reka = Hand()
reka.get_specific_cards(ta,['10h', '8d', '9h', '7C','jC'])
for i in reka.cards_in_hand:
    print(i)
print('**************')
print(len(ta))
reka.check_hand()
print(reka.in_hand)
print(str(set([card.colour for card in reka.cards_in_hand])))

błędne oznaczenie figury karty
błędne oznaczenie figury karty
błędne oznaczenie figury karty
błędne oznaczenie figury karty
błędne oznaczenie figury karty


KeyboardInterrupt: Interrupted by user

In [52]:
a = [1,2,3,4,5]
print([a[i+1] - a[i] for i in range(0,4)] == [1,1,1,1])


True


<p><i>małe testy ręki</i></p>

In [12]:
n_of_pairs = 0
n_of_two_pairs = 0
n_of_threes = 0
n_of_fours = 0
n = 2_000_000
t = Deck()
t.shuffle()
h1 = Hand()
for i in range(n):
    h1.get_card(t,5)
    h1.check_hand()
    if len(h1.in_hand['fours']) == 1:
        n_of_fours +=1
    elif len(h1.in_hand['threes']) == 1:
        n_of_threes +=1
    elif len(h1.in_hand['pairs']) == 2:
        n_of_two_pairs += 1
    elif len(h1.in_hand['pairs']) == 1:
        n_of_pairs += 1
    h1.reset_hand()
    t.reset()
print(f'prawdopodobieństwo wyciągnięcia pary wynosi     {n_of_pairs/n*100:.3f}%')
print(f'prawdopodobieństwo wyciągnięcia 2 par  wynosi   {n_of_two_pairs/n*100:.3f}%')
print(f'prawdopodobieństwo wyciągnięcia trójki wynosi   {n_of_threes/n*100:.3f}%')
print(f'prawdopodobieństwo wyciągnięcia karety wynosi   {n_of_fours/n*100:.3f}%')

prawdopodobieństwo wyciągnięcia pary wynosi     42.222%
prawdopodobieństwo wyciągnięcia 2 par  wynosi   4.747%
prawdopodobieństwo wyciągnięcia trójki wynosi   2.260%
prawdopodobieństwo wyciągnięcia karety wynosi   0.024%


class Player:
    def __init__(self, name = ''):        
        self.name = name
        self.balance = 0
        self.hand = Hand()
        self.hand_history = []
    def __repr__(self):
        return f"""  \n player name:{self.name} \n current $$ :{self.balance} \n cards in hand: {self.hand.cards_in_hand}\n high card :{self.hand.high_card}"""


class Game:
    def __init__(self, players, blind = 0):
        self.start_time = None  # to work on
        self.end_time   = None  #  as above
        self.list_of_players= [] 
        self.deck = Deck()
        self.deck.shuffle()

        for player in players:
            self.list_of_players.append(player)
        
        self.loot = blind
        self.blind_value = 0
        self.small_blind_value = self.blind_value//2
        self.loot = self.blind_value + self.small_blind_value

    def end_game(self):
        return True
